<a href="https://colab.research.google.com/github/ByungjunKim/LiterarySource/blob/main/ProjectGutenberg_WikiSource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Gutenberg / WikiSource

### 필요 패키지 설치 및 로드

In [ ]:
pip install -q gutenbergpy wikiextractor gdown

In [ ]:
# 필요 패키지 load
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import gutenbergpy.textget
from pprint import pprint
import gdown

### 구텐베르크 프로젝트
[Project Gutenberg](https://www.gutenberg.org/)  
[파이썬에서 구텐베르크 프로젝트 자료 자동 수집(gutenbergpy)](https://github.com/raduangelescu/gutenbergpy)

##### 메타정보
아래 사이트에서 자동으로 구텐베르크 프로젝트 전체 메타정보 업데이트 중  
https://github.com/hugovk/gutenberg-metadata

In [ ]:
# 메타정보 json 파일
# http://hugovk.github.io/gutenberg-metadata/gutenberg-metadata.json
meta = pd.read_json('http://hugovk.github.io/gutenberg-metadata/gutenberg-metadata.json',orient='index')
meta

In [ ]:
# index 컬럼 생성 (구텐베르크 프로젝트 고유번호와 일치)
meta = meta.reset_index()
meta

In [ ]:
# subject 컬럼은 무엇일까?
meta['subject'][104]

In [ ]:
# 제인 오스틴(Austen, Jane) 참여한 저작만 필터링한다면?
meta['author'].map(lambda x:'Austen, Jane' in x)

In [ ]:
# 연습 (jane이라는 변수로 데이터 프레임 만들기)
# 힌트 : 대괄호 활용하기


In [ ]:
# 연습
# 제인 오스틴(Austen, Jane) 혼자 쓴 필터링한다면?
# author에 제인 오스틴 혼자만 있는 경우


In [ ]:
# 연습 : subject에 love stories가 포함된 작품 뽑기

##### 서적

In [ ]:
# 서적 고유 id로 수집하기
def usage_example(id):
    # This gets a book by its gutenberg id number
    raw_book = gutenbergpy.textget.get_text_by_id(id) # with headers
    clean_book = gutenbergpy.textget.strip_headers(raw_book) # without headers
    return str(clean_book), str(raw_book)

In [ ]:
# 필경사 바틀비(11231)
# https://www.gutenberg.org/ebooks/11231
# 여러분이 수집하고 싶은 책 번호를 넣어보세요
cleaned_book, raw_book = usage_example(11231)

In [ ]:
# clean_book 과 raw_book의 차이는?

In [ ]:
# 각종 메타정보 포함한 버전(''The Project Gutenberg eBook of Bartleby, The Scrivener, by Herman Melvil ...')
pprint(raw_book)

In [ ]:
# 각종 메타 정보 삭제한 버전
pprint(cleaned_book)

##### 한 작가의 여러 서적 수집해서 엑셀로 저장하기

In [ ]:
# 제인 오스틴의 저작 index 리스트 만들기
print(jane['index'].tolist())
jane_index = jane['index'].tolist()

In [ ]:
# for문을 활용해서 여러 작품을 한꺼번에 수집해 데이터프레임으로 만들기
text_list = [] # 빈 list 만들기
for index in tqdm(jane_index[:10]): # 열개만 하기 전체하려면 [:10] 삭제
  try: # 에러 예외처리
    cleaned_book, raw_book = usage_example(index) # index 값을 넣는다.
    res = {'index':index,'cleaned_book':cleaned_book,'raw_book':raw_book}
    text_list.append(res)
  except: # 에러가 나면 아래를 코드를 실행해라.
    print(f'\n{index}') #에러나는 index를 print

In [ ]:
# 결과 확인
text_list[0]

In [ ]:
# 데이터 프레임으로 만들기
jane_text = pd.DataFrame(text_list)
jane_text

In [ ]:
# 메타 정보와 합치기 (index 열 기준으로)
jane_merge = jane.merge(jane_text, on ='index')
jane_merge

In [ ]:
# 연습 : csv로 저장


### 위키문헌 (wikisource)
https://dumps.wikimedia.org/kowikisource/20231001/

##### 위키문헌 전처리

In [ ]:
# 위키문헌 xml 원본 파일 다운로드
!wget https://dumps.wikimedia.org/kowikisource/20231001/kowikisource-20231001-pages-articles.xml.bz2

In [ ]:
# Parsing (3분 가량 걸림)
!python -m wikiextractor.WikiExtractor kowikisource-20231001-pages-articles.xml.bz2

In [ ]:
# 파싱한 파일을 하나의 xml로 합치기
!cat text/*/* > kowikisource.xml

In [ ]:
# 맨위에 추가
!echo "$(echo -n -e '<?xml version="1.0" encoding="UTF-8"?>\n<wikisource>\n'; cat kowikisource.xml)" > kowikisource.xml

In [ ]:
!head -n 20 kowikisource.xml # 첫 20줄만 확인

In [ ]:
# 맨 아랫줄에 추가
!echo '</wikisource>' >> kowikisource.xml

In [ ]:
!tail kowikisource.xml # 마지막줄 확인

##### 위키문헌 다루기

In [ ]:
df = pd.read_xml('./kowikisource.xml')
df

In [ ]:
# 소설가 정보 load
# https://docs.google.com/spreadsheets/d/1u9sB_HxpmKIYWvS898e6_z2_jb5eStvY/edit?usp=sharing&ouid=102864613966848491919&rtpof=true&sd=true
gdown.download(id='1u9sB_HxpmKIYWvS898e6_z2_jb5eStvY',output='230121_위키문헌_문학가.xlsx')

In [ ]:
article = pd.read_excel('./230121_위키문헌_문학가.xlsx')
article

In [ ]:
# 위키문헌 문학 제목
titles = article['title'].tolist()
len(titles) #649 행

In [ ]:
# merge
df = df.merge(article,on='title')
df

In [ ]:
# 연습: 엑셀로 저장하기
